In [10]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import compare_ssim as ssim

def mse (image1, image2):
    error = np.sum((image1.astype('float') - image2.astype('float'))**2)
    #Smaller is more similar
    error = error/float(image1.shape[0]*image1.shape[1])
    return error

def dssimCompare (image1, image2):
    #Smaller is more similar
    return (1 - ssim(image1, image2, 3))/2

def psnrCompare (image1, image2):
    #See http://docs.opencv.org/2.4/doc/tutorials/highgui/video-input-psnr-ssim/video-input-psnr-ssim.html#videoinputpsnrmssim
    #Larger is more similar!
    error = 10*log10((255*255)/mse(image1, image2))
    return error

def msssimCompare(image1, image2):
    return error
    
def plotError(frameCount):
    print "Now plotting errors"
    #Create our array of zeros as a 2D numpy array
    errors = np.zeros((frameCount, frameCount))
    #Now Populate our 2 Diagonal Errors
    for i in range(0, frameCount-1):
        image1 = cv2.imread("frame%0*d.png" %(4,i+1))
        image2 = cv2.imread("frame%0*d.png" %(4,i+2))
        #First Diagonal is y=x+1
        errors[i][i+1]=mse(image1, image2)
        print "hi %d" %errors[i][i+1]
    #Save our array
    np.savetxt('array.txt', errors)
    print "Array saved to disk"
    #Now plot this thing
    X, Y = np.meshgrid(x, y)
    Z = z.reshape(len (x), len (y))
    plt.pcolormesh(X, Y, Z)
    plt.title('DSSIM Value Between Frames')
    plt.grid(True)
    plt.savefig("similarityplot.png")
    plt.show()

#Main Function
plotError(165)


Now plotting errors
hi 7
hi 6
hi 4
hi 2
hi 2
hi 2
hi 3
hi 3
hi 3
hi 2
hi 1
hi 2
hi 2
hi 2
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 1
hi 4
hi 5
hi 92
hi 62
hi 61
hi 93
hi 52
hi 131
hi 131
hi 80
hi 129
hi 185
hi 211
hi 240
hi 275
hi 252
hi 195
hi 205
hi 215
hi 177
hi 186
hi 352
hi 230
hi 228
hi 157
hi 92
hi 80
hi 51
hi 67
hi 18
hi 5
hi 3
hi 23
hi 33
hi 35
hi 70
hi 120
hi 150
hi 205
hi 312
hi 289
hi 292
hi 322
hi 383
hi 446
hi 268
hi 271
hi 229
hi 225
hi 275
hi 468
hi 306
hi 221
hi 259
hi 187
hi 169
hi 110
hi 194
hi 294
hi 11
hi 5
hi 3
hi 3
hi 2
hi 2
hi 2
hi 2
hi 2
hi 1
hi 1
hi 2
hi 3
hi 2
hi 3
hi 3
hi 8
hi 2
hi 2
hi 2
hi 5
hi 5
hi 2
hi 2
hi 3
hi 2
hi 1
hi 1
hi 1
hi 2
hi 1
hi 1
hi 35
hi 12
hi 11
hi 8
hi 2
hi 2
hi 59
hi 6
hi 6
hi 65
hi 41
hi 83
hi 83
hi 82
hi 35
hi 63
hi 55
hi 57
hi 31
hi 44
hi 140
hi 41
hi 30
hi 28
hi 9
hi 32
hi 21
hi 36
hi 16
hi 20
hi 2
hi 23
hi 33
hi 69
hi 60
hi 45


NameError: global name 'x' is not defined